In [19]:
# parse_html_and_json_products.py

import json, re
from pathlib import Path
from urllib.parse import urljoin


HTML_PATHS = sorted(Path(".").glob("page*.html"))
JSON_FILES = sorted(Path(".").glob("page*.json"))
OUT_CSV = Path("sephora_products.csv")


BASE = "https://www.sephora.com"

def to_float(x):
    if x is None: return None
    s = str(x).strip().replace(",", "")
    if s.startswith("$"): s = s[1:]
    try: return float(s)
    except ValueError: return None

def to_int(x):
    if x is None: return None
    try: return int(str(x).replace(",", "").strip())
    except ValueError: return None

def first(*vals):
    for v in vals:
        if v not in (None, "", [], {}):
            return v
    return None

def norm_url(u):
    if not u: return None
    return urljoin(BASE, u)

# ---------- A) Extract product dicts embedded in the HTML text ----------
def extract_inline_products_from_html(html_text: str):
    """
    Finds JSON objects that look like products embedded in HTML/JS.
    We scan for '{"brandName"' starts and then brace-match to the closing '}'.
    """
    objs = []
    needle = '{"brandName"'
    i = 0
    n = len(html_text)
    while True:
        start = html_text.find(needle, i)
        if start == -1:
            break
        # back up one char if there's a leading '{' before the needle
        # (our needle already starts with '{', so 'start' is at the '{')
        brace_count = 0
        j = start
        in_str = False
        esc = False
        # brace-match until we close the top-level object
        while j < n:
            ch = html_text[j]
            if in_str:
                if esc:
                    esc = False
                elif ch == '\\':
                    esc = True
                elif ch == '"':
                    in_str = False
            else:
                if ch == '"':
                    in_str = True
                elif ch == '{':
                    brace_count += 1
                elif ch == '}':
                    brace_count -= 1
                    if brace_count == 0:
                        # end of object
                        block = html_text[start:j+1]
                        # try to load JSON
                        try:
                            obj = json.loads(block)
                            objs.append(obj)
                        except Exception:
                            pass
                        i = j + 1
                        break
            j += 1
        else:
            # ran out of text
            break
    return objs

# ---------- B) Walk any JSON structure to yield product-like dicts ----------
def walk_products(node):
    """
    Yield dicts that look like product/sku records anywhere in JSON.
    """
    if isinstance(node, dict):
        if any(k in node for k in ("productId","displayName","productName","currentSku","brandName","skuId")):
            yield node
        for v in node.values():
            yield from walk_products(v)
    elif isinstance(node, list):
        for v in node:
            yield from walk_products(v)

# ---------- C) Normalize to tidy rows ----------
def normalize(prod: dict):
    # IDs
    product_id = first(prod.get("productId"), prod.get("id"))
    current_sku = prod.get("currentSku") or {}
    sku_id = first(prod.get("skuId"),
                   (prod.get("sku") or {}).get("skuId"),
                   current_sku.get("skuId"))
    # Names
    product_name = first(prod.get("displayName"), prod.get("productName"), prod.get("name"))
    # Brand
    brand = prod.get("brand")
    brand_name = first(prod.get("brandName"),
                       (brand or {}).get("brandName"),
                       (brand or {}).get("name"))
    # Prices
    list_price = to_float(first(prod.get("listPrice"),
                                current_sku.get("listPrice"),
                                (prod.get("price") or {}).get("listPrice"),
                                (prod.get("price") or {}).get("formattedPrice")))
    sale_price = to_float(first(prod.get("salePrice"),
                                current_sku.get("salePrice"),
                                (prod.get("price") or {}).get("salePrice")))
    # Rating / reviews
    rating = to_float(first(prod.get("rating"),
                            prod.get("starRatings"),
                            prod.get("reviewRating")))
    reviews = to_int(first(prod.get("reviews"),
                           prod.get("reviewsCount"),
                           prod.get("reviewCount")))
    # URLs
    url = first(prod.get("targetUrl"), prod.get("productUrl"),
                prod.get("canonicalUrl"), prod.get("targetURL"))
    if url and isinstance(url, str) and url.startswith("/"):
        url = norm_url(url)
    if not url and isinstance(product_id, str) and product_id.startswith("P"):
        url = f"{BASE}/product/{product_id}"
    # Images
    image_url = first(
        prod.get("heroImage"),
        prod.get("altImage"),
        (prod.get("skuImages") or {}).get("imageUrl") if isinstance(prod.get("skuImages"), dict) else None,
        (current_sku.get("skuImage") or {}).get("imageUrl"),
        (prod.get("image") or {}).get("src"),
        (prod.get("image") or {}).get("url"),
        (prod.get("images")[0].get("src") if isinstance(prod.get("images"), list) and prod["images"] else None)
    )
    return {
        "skuId": sku_id,
        "productId": product_id,
        "brandName": brand_name,
        "productName": product_name,
        "listPrice": list_price,
        "salePrice": sale_price,
        "rating": rating,
        "reviews": reviews,
        "imageUrl": image_url,
        "url": url,
        "isNew": bool(current_sku.get("isNew", False)),
        "isLimitedEdition": bool(current_sku.get("isLimitedEdition", False)),
        "isSephoraExclusive": bool(current_sku.get("isSephoraExclusive", False)),
    }

# ---------- D) Load everything, dedupe, and export ----------
all_nodes = []

# 1) HTML inline products
if HTML_PATH.exists():
    html_text = HTML_PATH.read_text(encoding="utf-8", errors="ignore")
    inline = extract_inline_products_from_html(html_text)
    all_nodes.extend(inline)
else:
    print(f"Warning: HTML not found: {HTML_PATH}")

# 2) JSON files (page2.json, page3.json, etc.)
for jf in JSON_FILES:
    if jf.exists():
        try:
            data = json.loads(jf.read_text(encoding="utf-8"))
            all_nodes.extend(walk_products(data))
        except Exception as e:
            print(f"Skipping {jf.name} (JSON load error): {e}")
    else:
        print(f"Warning: JSON not found: {jf}")

# 3) Normalize + dedupe by skuId (fallback to productId)
rows, seen = [], set()
for node in all_nodes:
    if not isinstance(node, dict):
        continue
    row = normalize(node)
    key = row["skuId"] or (f"PID:{row['productId']}" if row["productId"] else None)
    # require at least name or brand to avoid noise
    if not key or not (row["productName"] or row["brandName"]):
        continue
    if key in seen:
        continue
    seen.add(key)
    rows.append(row)

print(f"Parsed {len(rows)} unique products.")
for r in rows[:10]:
    print("-", r["brandName"], "|", r["productName"], "|", r["listPrice"], "|", r["url"])

# 4) Save CSV
if rows:
    import csv
    OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
    with OUT_CSV.open("w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=rows[0].keys())
        w.writeheader()
        w.writerows(rows)
    print("Wrote:", OUT_CSV)



Parsed 144 unique products.
- SEPHORA COLLECTION | Cream Lip Stain 10HR Liquid Lipstick | 16.0 | https://www.sephora.com/product/cream-lip-stain-liquid-lipstick-P281411?skuId=2760981
- Anastasia Beverly Hills | Full-Pigment Matte & Satin Velvet Lipstick | 26.0 | https://www.sephora.com/product/anastasia-beverly-hills-matte-satin-velvet-lipstick-P480576?skuId=2882280
- SEPHORA COLLECTION | Satin Hydrating Lipstick | 16.0 | https://www.sephora.com/product/satin-hydrating-lipstick-P501496?skuId=2564474
- Fenty Beauty by Rihanna | Gloss Bomb Stix High-Shine Gloss Stick | 26.0 | https://www.sephora.com/product/fenty-beauty-rihanna-gloss-bomb-stix-high-shine-gloss-stick-P511572?skuId=2787497
- SEPHORA COLLECTION | Matte Velvet Lipstick | None | https://www.sephora.com/product/matte-velvet-lipstick-P506548?skuId=2666840
- MERIT | Signature Lip Lightweight Lipstick | 26.0 | https://www.sephora.com/product/merit-signature-lip-lightweight-lipstick-P481403?skuId=2792802
- DIOR | Dior Addict Shine